In [1]:
import numpy as np
import matplotlib.pyplot as plt

In this Python notebook, the code of http://ieeexplore.ieee.org/document/996017/ is (tried to be implemented

In [10]:
from pygmo import *
algo = algorithm(nsga2(gen=10))
algo.set_verbosity(20)
pop = population(zdt(1), 40)

In [11]:
pop = algo.evolve(pop) 
uda = algo.extract(nsga2)
uda.get_log() 

[(1, 0, array([ 0.00443894,  2.59065932]))]